In [4]:
import numpy as np
import pandas as pd

In [5]:
sentiment = pd.read_csv("data/tweets_sentiment.csv")
sentiment.drop(sentiment.columns[[0]], axis=1, inplace=True)
sentiment.rename(columns = {'created_at':'Date'}, inplace = True)
spy = pd.read_csv("data/SPY.csv")

In [6]:
df = pd.merge(sentiment, spy, on ='Date', how ="inner")
df.head()

,Date,pos_score,neu_score,neg_score,compound,Open,High,Low,Close,Adj Close,Volume
0,2020-04-09,0.073174,0.887866,0.038960,0.114438,277.579987,281.200012,275.470001,278.200012,271.092224,189999200
1,2020-04-13,0.076010,0.887225,0.036771,0.122273,277.140015,277.510010,271.410004,275.660004,268.617126,114839100
2,2020-04-14,0.083978,0.871450,0.044567,0.136125,280.980011,284.899994,275.510010,283.790009,276.539368,134143400
3,2020-04-15,0.083737,0.875221,0.041041,0.152802,277.570007,283.940002,275.459991,277.760010,270.663422,121775000
4,2020-04-16,0.081858,0.878151,0.039988,0.127939,279.149994,280.029999,275.760010,279.100006,271.969208,131798300


In [12]:
opn = {'Date': [df['Date'][i] for i in range(3,len(df))], \
       'Movement': [1 if df['Close'][i] - df['Close'][i-1] > 0 else 0 for i in range(3,len(df))], \
       'Close1': [df['Close'][i-1] for i in range(3,len(df))], \
       'Close2': [df['Close'][i-2] for i in range(3,len(df))], \
       'Close3': [df['Close'][i-3] for i in range(3,len(df))]}

sen = {'Date': [df['Date'][i] for i in range(3,len(df))], \
       'Sentiment1': [df['compound'][i-1] for i in range(3,len(df))], \
       'Sentiment2': [df['compound'][i-2] for i in range(3,len(df))], \
       'Sentiment3': [df['compound'][i-3] for i in range(3,len(df))]}

In [13]:
o = pd.DataFrame(opn)
s = pd.DataFrame(sen)
data = pd.merge(o, s, on='Date', how='inner')

In [14]:
data.head()

,Date,Movement,Close1,Close2,Close3,Sentiment1,Sentiment2,Sentiment3
0,2020-04-15,0,283.790009,275.660004,278.200012,0.136125,0.122273,0.114438
1,2020-04-16,1,277.760010,283.790009,275.660004,0.152802,0.136125,0.122273
2,2020-04-17,1,279.100006,277.760010,283.790009,0.127939,0.152802,0.136125
3,2020-04-20,0,286.640015,279.100006,277.760010,0.130096,0.127939,0.152802
4,2020-04-21,0,281.589996,286.640015,279.100006,0.109606,0.130096,0.127939


## Training Model

In [15]:
from sklearn.model_selection import train_test_split
X = data[data.columns[2:]]
y = data[data.columns[1]]
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3)

In [16]:
positives = sum([1 if y[i] == 1 else 0 for i in range(len(y))])
negatives = len(y) - positives
print("Positive: ", positives)
print("Negative: ", negatives)

Positive:  31
Negative:  19


In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

### Linear Kernel

In [18]:
from sklearn.svm import SVC
classifier = SVC(kernel='linear',verbose=True)
classifier.fit(X_train, y_train)

[LibSVM].
*.*
optimization finished, #iter = 92
obj = -25.957926, rho = 0.931621
nSV = 27, nBSV = 22
Total nSV = 27


SVC(kernel='linear', verbose=True)

In [19]:
y_pred = classifier.predict(X_test)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred));

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.60      1.00      0.75         9

    accuracy                           0.60        15
   macro avg       0.30      0.50      0.37        15
weighted avg       0.36      0.60      0.45        15



/Users/zariski/mambaforge/envs/vip/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zariski/mambaforge/envs/vip/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/zariski/mambaforge/envs/vip/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

### RBF Kernel

In [21]:
classifier = SVC(kernel='rbf',verbose=True)
classifier.fit(X_train, y_train)

[LibSVM]

SVC(verbose=True)

.*
optimization finished, #iter = 57
obj = -24.114751, rho = 0.775358
nSV = 31, nBSV = 18
Total nSV = 31


In [22]:
y_pred = classifier.predict(X_test)

In [23]:
print(classification_report(y_test, y_pred));

              precision    recall  f1-score   support

           0       1.00      0.17      0.29         6
           1       0.64      1.00      0.78         9

    accuracy                           0.67        15
   macro avg       0.82      0.58      0.53        15
weighted avg       0.79      0.67      0.58        15



### Polynomial Kernel

In [24]:
classifier = SVC(kernel='poly',verbose=True)
classifier.fit(X_train, y_train)

[LibSVM]

SVC(kernel='poly', verbose=True)

..*...*
optimization finished, #iter = 177
obj = -22.951944, rho = 0.901956
nSV = 30, nBSV = 20
Total nSV = 30


In [25]:
y_pred = classifier.predict(X_test)

In [26]:
print(classification_report(y_test, y_pred));

              precision    recall  f1-score   support

           0       1.00      0.17      0.29         6
           1       0.64      1.00      0.78         9

    accuracy                           0.67        15
   macro avg       0.82      0.58      0.53        15
weighted avg       0.79      0.67      0.58        15



### Sigmoid Kernel

In [27]:
classifier = SVC(kernel='sigmoid',verbose=True)
classifier.fit(X_train, y_train)

[LibSVM]

SVC(kernel='sigmoid', verbose=True)

*
optimization finished, #iter = 29
obj = -27.637876, rho = 0.865379
nSV = 28, nBSV = 24
Total nSV = 28


In [28]:
y_pred = classifier.predict(X_test)

In [29]:
print(classification_report(y_test, y_pred));

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         6
           1       0.57      0.89      0.70         9

    accuracy                           0.53        15
   macro avg       0.29      0.44      0.35        15
weighted avg       0.34      0.53      0.42        15

